In [103]:
#Question 6.1b
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

house_df = pd.read_csv('BostonHousing.csv')
house_df = house_df.iloc[0:1000]
predictors = ['CRIM', 'CHAS', 'RM']
outcome = 'MEDV'

X = pd.get_dummies(house_df[predictors], drop_first=True)
y = house_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

house_lm = LinearRegression()
house_lm.fit(train_X, train_y)

print(pd.DataFrame({'Predicted': ['Intercept'] + predictors, 'Coefficient': [house_lm.intercept_] + list(house_lm.coef_)}))

   Predicted  Coefficient
0  Intercept   -29.193467
1       CRIM    -0.240062
2       CHAS     3.266817
3         RM     8.325175


In [87]:
#Question 6.1c
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

house_df = pd.read_csv('BostonHousing.csv')

predictors = ['CRIM', 'CHAS', 'RM']
response = 'MEDV'

train_X, valid_X, train_y, valid_y = train_test_split(house_df[predictors], house_df[response], test_size=0.4, random_state=1)

model = LinearRegression()
model.fit(train_X, train_y)

data = pd.DataFrame({'CRIM': [0.1], 'CHAS': [0], 'RM': [6]})
price = model.predict(data)

print(f"Predicted median house price: ${price[0]*1000}")

Predicted median house price: $20733.578132519167


In [3]:
#Question 6.1d(ii)
import pandas as pd

house_df = pd.read_csv('BostonHousing.csv')

predictors = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']
response = 'MEDV'

matrix = house_df[predictors].corr()
print(matrix)

             CRIM        ZN     INDUS      CHAS       NOX        RM       AGE  \
CRIM     1.000000 -0.200469  0.406583 -0.055892  0.420972 -0.219247  0.352734   
ZN      -0.200469  1.000000 -0.533828 -0.042697 -0.516604  0.311991 -0.569537   
INDUS    0.406583 -0.533828  1.000000  0.062938  0.763651 -0.391676  0.644779   
CHAS    -0.055892 -0.042697  0.062938  1.000000  0.091203  0.091251  0.086518   
NOX      0.420972 -0.516604  0.763651  0.091203  1.000000 -0.302188  0.731470   
RM      -0.219247  0.311991 -0.391676  0.091251 -0.302188  1.000000 -0.240265   
AGE      0.352734 -0.569537  0.644779  0.086518  0.731470 -0.240265  1.000000   
DIS     -0.379670  0.664408 -0.708027 -0.099176 -0.769230  0.205246 -0.747881   
RAD      0.625505 -0.311948  0.595129 -0.007368  0.611441 -0.209847  0.456022   
TAX      0.582764 -0.314563  0.720760 -0.035587  0.668023 -0.292048  0.506456   
PTRATIO  0.289946 -0.391679  0.383248 -0.121515  0.188933 -0.355501  0.261515   
LSTAT    0.455621 -0.412995 

In [9]:
#Question 6.1d(iii)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
from dmba import regressionSummary, exhaustive_search, backward_elimination, forward_selection, stepwise_selection, adjusted_r2_score, AIC_score, BIC_score

house_df = pd.read_csv('BostonHousing.csv')

predictors = ['INDUS', 'NOX', 'RM', 'TAX', 'PTRATIO']
response = 'MEDV'
train_X, valid_X, train_y, valid_y = train_test_split(house_df[predictors], house_df[response], test_size=0.4, random_state=1)

# Helper
def evaluate_model(model, valid_X, valid_y):
    predictions = model.predict(valid_X)
    residuals = valid_y - predictions
    rmse = mean_squared_error(valid_y, predictions, squared=False)
    mape = mean_absolute_percentage_error(valid_y, predictions)
    mean_error = residuals.mean()
    return rmse, mape, mean_error, residuals

def plot_histogram(residuals, model_name):
    plt.hist(residuals, bins=20, edgecolor='black', alpha=0.7)
    plt.title(f'Residual Histogram: {model_name}')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.savefig(f'{model_name}_histogram.jpg')
    plt.close()


#Backward
print("Backward Elimination")
def train_model(variables):
  model = LinearRegression()
  model.fit(train_X[variables], train_y)
  return model

def score_model(model, variables):
  return AIC_score(train_y, model.predict(train_X[variables]), model)

allVariables = train_X.columns
best_model, best_variables = backward_elimination(allVariables, train_model, score_model, verbose=True)

print(best_variables)
rmse_backward, mape_backward, mean_error_backward, residuals_backward = evaluate_model(best_model, valid_X[best_variables], valid_y)
print(f"Backward Elimination - RMSE: {rmse_backward}, MAPE: {mape_backward}, Mean Error: {mean_error_backward}")
plot_histogram(residuals_backward, 'Backward Elimination')
print()
print()

#Forward
print("Forward Elimination")
def train_model(variables):
    if len(variables) == 0: 
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)
    
best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)
print(best_variables)
rmse_forward, mape_forward, mean_error_forward, residuals_forward = evaluate_model(best_model, valid_X[best_variables], valid_y)
print(f"Forward Selection - RMSE: {rmse_forward}, MAPE: {mape_forward}, Mean Error: {mean_error_forward}")
plot_histogram(residuals_forward, 'Forward Selection')
print()
print()

#Stepwise
print("Stepwise Elimination Initial")
def train_model(variables):
    if len(variables) == 0: 
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)
    
best_model, best_variables = stepwise_selection(train_X.columns, train_model, score_model, verbose=True)
print(best_variables)
rmse_stepwise, mape_stepwise, mean_error_stepwise, residuals_stepwise = evaluate_model(best_model, valid_X[best_variables], valid_y)
print(f"Stepwise Selection - RMSE: {rmse_stepwise}, MAPE: {mape_stepwise}, Mean Error: {mean_error_stepwise}")
plot_histogram(residuals_stepwise, 'Stepwise Selection Initial')
print()
print()

print("Stepwise Elimination Final")
predictors = ['INDUS', 'NOX', 'TAX']
response = 'MEDV'
train_X, valid_X, train_y, valid_y = train_test_split(house_df[predictors], house_df[response], test_size=0.4, random_state=1)

model = LinearRegression()
model.fit(train_X, train_y)

def train_model(variables):
    if len(variables) == 0: 
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)
    
best_model, best_variables = stepwise_selection(train_X.columns, train_model, score_model, verbose=True)
print(best_variables)
rmse_stepwise, mape_stepwise, mean_error_stepwise, residuals_stepwise = evaluate_model(best_model, valid_X[best_variables], valid_y)
print(f"Stepwise Selection - RMSE: {rmse_stepwise}, MAPE: {mape_stepwise}, Mean Error: {mean_error_stepwise}")
plot_histogram(residuals_stepwise, 'Stepwise Selection Final')
print()
print()


Backward Elimination
Variables: INDUS, NOX, RM, TAX, PTRATIO
Start: score=1907.87
Step: score=1905.87, remove INDUS
Step: score=1905.16, remove TAX
Step: score=1905.16, remove None
['NOX', 'RM', 'PTRATIO']
Backward Elimination - RMSE: 6.195415103912311, MAPE: 0.2059143069398552, Mean Error: 0.41950040137259326


Forward Elimination
Variables: INDUS, NOX, RM, TAX, PTRATIO
Start: score=2191.75, constant
Step: score=1989.28, add RM
Step: score=1938.72, add TAX
Step: score=1921.51, add PTRATIO
Step: score=1905.87, add NOX
Step: score=1905.87, add None
['RM', 'TAX', 'PTRATIO', 'NOX']
Forward Selection - RMSE: 6.151954158615837, MAPE: 0.20168775708472764, Mean Error: 0.4024606963885238


Stepwise Elimination Initial
Variables: INDUS, NOX, RM, TAX, PTRATIO
Start: score=2191.75, constant
Step: score=1989.28, add RM
Step: score=1938.72, add TAX
Step: score=1921.51, add PTRATIO
Step: score=1905.87, add NOX
Step: score=1905.16, remove TAX
Step: score=1905.16, unchanged None
['RM', 'PTRATIO', 'NOX

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(




Stepwise Elimination Final
Variables: INDUS, NOX, TAX
Start: score=2191.75, constant
Step: score=2096.15, add INDUS
Step: score=2090.93, add TAX
Step: score=2088.35, add NOX
Step: score=2088.35, unchanged None
['INDUS', 'TAX', 'NOX']
Stepwise Selection - RMSE: 8.515131003401658, MAPE: 0.2525132755692878, Mean Error: 0.8923224390011274




/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [117]:
#Question 7.3a
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
import matplotlib.pylab as plt
import numpy as np

data = pd.read_csv('BostonHousing.csv')

X = data.drop(columns=["MEDV", "CAT. MEDV"])
y = data["MEDV"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=1)

scaler = preprocessing.StandardScaler()
X_train_new = scaler.fit_transform(X_train)
X_val_new = scaler.transform(X_val)

mse_results = {}
for k in range(1, 6):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_new, y_train)
    y_val_pred = knn.predict(X_val_new)
    mse = mean_squared_error(y_val, y_val_pred)
    mse_results[k] = mse

best_k = min(mse_results, key=mse_results.get)

print(f"Best k: {best_k}, MSE: {mse_results[best_k]}")

new_data_point = pd.DataFrame({
    'CRIM': [0.2],
    'ZN': [0],
    'INDUS': [7],
    'CHAS': [0],
    'NOX': [0.538],
    'RM': [6],
    'AGE': [62],
    'DIS': [4.7],
    'RAD': [4],
    'TAX': [307],
    'PTRATIO': [21],
    'LSTAT': [10]
})

new_data_point_scaled = scaler.transform(new_data_point)

knn = KNeighborsRegressor(n_neighbors=best_k)
knn.fit(X_train_new, y_train)
medv_prediction = knn.predict(new_data_point_scaled)

print(f"Predicted MEDV: {medv_prediction[0]}")

Best k: 3, MSE: 21.82572523262178
Predicted MEDV: 18.766666666666666


In [96]:
#Question 7.3bc
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt


data = pd.read_csv('BostonHousing.csv')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=1)

scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

mse_results = {}
for k in range(1, 6):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    y_val_pred = knn.predict(X_val_scaled)
    
    mse = mean_squared_error(y_val, y_val_pred)
 
    mse_results[k] = mse

#MSE for k = 1 to 5
print("MSE: ",mse_results)

data_scaled = scaler.transform(new_data_point)

knn_best = KNeighborsRegressor(n_neighbors=3)
knn_best.fit(X_train_scaled, y_train)
medv_prediction = knn_best.predict(new_data_point_scaled)

print("MEDV Prediction, k = 3: $",medv_prediction[0])

y_train_pred = knn_best.predict(X_train_scaled)

train_mse = mean_squared_error(y_train, y_train_pred)
print("MSE: ",train_mse)  

MSE:  {1: 29.194876847290637, 2: 22.83465517241379, 3: 21.82572523262178, 4: 22.936616379310347, 5: 25.14845123152709}
MEDV Prediction, k = 3: $ 18.766666666666666
MSE:  11.347334066740007


In [143]:
#Question 8.2abc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

data = pd.read_csv('accidentsFull.csv')

#a
data['INJURY'] = data['MAX_SEV_IR'].apply(lambda x: 'yes' if x in [1, 2] else 'no')
prediction = data['INJURY'].value_counts().idxmax()
print("Part a:")
print(f"The baseline prediction should be: INJURY = {prediction}")
print()


#b(i)
subset = data[['INJURY', 'WEATHER_R', 'TRAF_CON_R']].head(12)
pivot_table = pd.pivot_table(subset, values='INJURY', index=['WEATHER_R', 'TRAF_CON_R'], columns=['INJURY'], aggfunc=len, fill_value=0)
print("Part b(i):")
print(pivot_table)
print()

#b(ii)
combination_counts = subset.groupby(['WEATHER_R', 'TRAF_CON_R', 'INJURY']).size().unstack(fill_value=0)
combination_counts['P(INJURY = yes)'] = combination_counts['yes'] / (combination_counts['yes'] + combination_counts['no'])
print("Part b(ii):")
print(combination_counts[['no', 'yes', 'P(INJURY = yes)']])
print()

#b(iii)
combination_counts['Prediction'] = combination_counts['P(INJURY = yes)'].apply(lambda x: 'yes' if x > 0.5 else 'no')
print("Part b(iii):")
print(combination_counts[['P(INJURY = yes)', 'Prediction']])
print()

#b(iv)
total_records = len(subset)
p_injury_yes = sum(subset['INJURY'] == 'yes') / total_records
p_weather_1_given_injury_yes = sum((subset['WEATHER_R'] == 1) & (subset['INJURY'] == 'yes')) / sum(subset['INJURY'] == 'yes')
p_traf_con_1_given_injury_yes = sum((subset['TRAF_CON_R'] == 1) & (subset['INJURY'] == 'yes')) / sum(subset['INJURY'] == 'yes')
p_weather_1 = sum(subset['WEATHER_R'] == 1) / total_records
p_traf_con_1 = sum(subset['TRAF_CON_R'] == 1) / total_records
naive_bayes_prob = (p_weather_1_given_injury_yes * p_traf_con_1_given_injury_yes * p_injury_yes) / (p_weather_1 * p_traf_con_1)
print(f"Part b(iv): {naive_bayes_prob:.4f}")
print()


#c(i)
data['INJURY'] = data['MAX_SEV_IR'].apply(lambda x: 'yes' if x in [1, 2] else 'no')
predictors = ['WRK_ZONE', 'WKDY_I_R', 'INT_HWY', 'PED_ACC_R', 'SPD_LIM', 'TRAF_CON_R', 'VEH_INVL', 'WEATHER_R']
outcome = 'INJURY'
X = pd.get_dummies(data[predictors])
y = data[outcome].astype('category')

#c(ii)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=1)
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_val_pred = nb_model.predict(X_val)
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Part c(ii)):")
print(conf_matrix)
print()

#c(iii)
accuracy = accuracy_score(y_val, y_val_pred)
error_rate = 1 - accuracy
print(f"Part c(iii): Overall error rate for the validation set: {error_rate:.2f}")
print()

#c(iv)
naive_prediction = y_val.mode()[0]
naive_error_rate = (y_val != naive_prediction).mean()
print(f"Part c(iv): Naive error rate: {naive_error_rate:.2f}")
percent_improvement = ((naive_error_rate - error_rate) / naive_error_rate) * 100
print(f"Percent improvement relative to the naive rule: {percent_improvement:.2f}%")

Part a:
The baseline prediction should be: INJURY = yes

Part b(i):
INJURY                no  yes
WEATHER_R TRAF_CON_R         
1         0            1    2
          1            1    0
          2            1    0
2         0            5    1
          1            1    0

Part b(ii):
INJURY                no  yes  P(INJURY = yes)
WEATHER_R TRAF_CON_R                          
1         0            1    2         0.666667
          1            1    0         0.000000
          2            1    0         0.000000
2         0            5    1         0.166667
          1            1    0         0.000000

Part b(iii):
INJURY                P(INJURY = yes) Prediction
WEATHER_R TRAF_CON_R                            
1         0                  0.666667        yes
          1                  0.000000         no
          2                  0.000000         no
2         0                  0.166667         no
          1                  0.000000         no

Part b(iv): 0.0000

Pa